In [1]:
import pandas as pd
import matplotlib as plt
import glob, os
import csv

In [2]:
test = pd.read_csv("201904-capitalbikeshare-tripdata.csv")
test.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,313,4/1/19 0:03,4/1/19 0:08,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W24063,Member
1,453,4/1/19 0:07,4/1/19 0:15,31119,14th & Belmont St NW,31298,15th & M St NW,W22080,Member
2,333,4/1/19 0:09,4/1/19 0:15,31203,14th & Rhode Island Ave NW,31519,1st & O St NW,W00276,Member
3,568,4/1/19 0:10,4/1/19 0:19,31202,14th & R St NW,31103,16th & Harvard St NW,W21582,Member
4,1609,4/1/19 0:10,4/1/19 0:37,31517,Union Market,31655,New Jersey Ave & F St NW,75558,Member


In [3]:
path = "/Users/germanportes/Documents/GW_Bootcamp/Project/Group-Project_team3/Resources"
path

'/Users/germanportes/Documents/GW_Bootcamp/Project/Group-Project_team3/Resources'

In [4]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
bikeshare= pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
bikeshare.to_csv( "Bikeshare_data.csv", index=False, encoding='utf-8')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,4,6,7,8,9,10,11,12,13,15,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
